In [1]:
import ipynb
import torch
from ipynb.fs.full.data_handler import load_subject, extract_labels, extract_action_interval, split_data, split_info, to_device
from ipynb.fs.full.train_model import train_model, accuracy_check

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

data, description = load_subject(1)
labels = extract_labels(description)
data_interval = extract_action_interval(data)

train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data_interval, labels)

train_data, val_data, test_data, train_labels, val_labels, test_labels = to_device(train_data, val_data, test_data, train_labels, val_labels, test_labels, device)
split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

[99, 99, 99, 99] 
 [13, 13, 14, 14] 
 [13, 13, 12, 12]
25.0 %  25.0 %  25.0 %  25.0 % 
24.074074074074073 %  24.074074074074073 %  25.925925925925924 %  25.925925925925924 % 
26.0 %  26.0 %  24.0 %  24.0 % 


# Deep CNN
Implementation of the deep CNN structure from (Schirrmeister et. al.) for the use on "Thinking out loud" dataset.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

class DeepCNN(nn.Module):
    def __init__(self, interval = "full", dropout = 0.5):
        super(DeepCNN, self).__init__()
        first_channels = 25
        second_channels = 50
        third_channels = 100
        fourth_channels = 200

        ### Block 1
        # Temporal convolution
        self.tempconv = nn.Conv2d(in_channels = 1, out_channels = first_channels, kernel_size = (1, 10), padding = 0, bias = False)
        # Spatial convolution
        self.spatconv = nn.Conv2d(in_channels = first_channels, out_channels = first_channels, kernel_size=(128,1), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm1 = nn.BatchNorm2d(first_channels, False)
        # ELU
        self.elu = nn.ELU()
        # Dropout
        self.dropout = nn.Dropout(dropout)
        # Max pooling
        self.maxpool = nn.MaxPool2d(kernel_size = (1,3), stride = (1,3))

        ### Block 2
        self.conv2 = nn.Conv2d(in_channels = first_channels, out_channels = second_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm2 = nn.BatchNorm2d(second_channels, False)

        ### Block 3
        self.conv3 = nn.Conv2d(in_channels = second_channels, out_channels = third_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm3 = nn.BatchNorm2d(third_channels, False)

        ### Block 4
        self.conv4 = nn.Conv2d(in_channels = third_channels, out_channels = fourth_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm4 = nn.BatchNorm2d(fourth_channels, False)

        # Classifier
        if interval == "action":
            self.classifier = nn.Linear(600,4, bias = False)
        elif interval == "full":
            self.classifier = nn.Linear(1800,4, bias = False)
        # Softmax
        self.softmax = nn.LogSoftmax(dim = 1)

        
    def forward(self, x):
        # Block 1
        res = self.tempconv(x)
        res = self.spatconv(res)
        res = self.batchnorm1(res)
        res = self.elu(res)
        res = self.maxpool(res)
        res = self.dropout(res)
        # Block 2
        res = self.conv2(res)
        res = self.batchnorm2(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 3
        res = self.conv3(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 4
        res = self.conv4(res)
        res = self.batchnorm4(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Classifier
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res

In [15]:
network = DeepCNN(interval = "action").float().to(device)
accuracy_check(network, train_data, train_labels)

print("####### TRAINING #######")
op = optim.Adam(params = network.parameters(), lr = 0.0001)
lossf = nn.NLLLoss()
train_model(network, train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 50, batch_size = 10, loss_func = lossf, optimizer = op)

accuracy_check(network, train_data, train_labels)

ACCURACY: 0.22727272727272727
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  1.4749458508613782 	 1.5506104230880737
1 	  1.299348880083133 	 1.492534875869751
2 	  1.2308249840369592 	 1.5194405317306519
3 	  1.1974896161984174 	 1.5659170150756836
4 	  1.1227928552872095 	 1.5262235403060913
5 	  1.0629359514285357 	 1.5007268190383911
6 	  0.9983711242675781 	 1.5426032543182373
7 	  0.9429540389623398 	 1.5290433168411255
8 	  0.8865502186310596 	 1.5421059131622314
9 	  0.8258804908165565 	 1.6197738647460938
10 	  0.7700865329840244 	 1.615899920463562
11 	  0.6795728145501553 	 1.6667762994766235
12 	  0.6641608996269031 	 1.6220803260803223
13 	  0.6184501647949219 	 1.7550822496414185
14 	  0.5887842911940354 	 1.7829394340515137
15 	  0.547271973047501 	 1.780473232269287
16 	  0.4680644793388171 	 1.8363385200500488
17 	  0.49186109885191304 	 1.873599886894226
18 	  0.4240741240672576 	 1.9217591285705566
19 	  0.3777558986957257 	 1.854255199432373
20 	  

In [16]:
accuracy_check(network, train_data, train_labels)
accuracy_check(network, val_data, val_labels)
accuracy_check(network, test_data, test_labels)


ACCURACY: 0.9848484848484849
ACCURACY: 0.3888888888888889
ACCURACY: 0.32
